<a href="https://colab.research.google.com/github/errec-sun/brain/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%A4%D0%B5%D0%B2%D1%80%D0%B0%D0%BB%D1%8C%D1%81%D0%BA%D0%B8%D0%B9_%D0%BA%D1%83%D1%80%D1%81_PRO_%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5_18_%D0%A1%D0%B5%D0%B3%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D1%8F_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Сегментация договоров при помощи архитектур: **Линейная Conv1D-сеть**, **PSPnet**, **UNET**

In [ ]:
!pip install pymorphy2 # установка лемматизатора pymorphy2

     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 7.1MB 4.5MB/s 


# Подключение библиотек

In [ ]:
import numpy as np # Библиотека для работы с массивами данных
from tensorflow.keras.models import Model, load_model # Импортируем модель, метод загрузки предобученной сети 
import re # Импортируем для работы со строками
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами и преобразования их в последовательности
# Импортируем слои для сеток
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Lambda
from tensorflow.keras import backend as K # Импортируем для подсчета dice_coef
from tensorflow.keras.optimizers import Adam, Adadelta # Импортируем оптимизаторы 
from tensorflow.keras import utils # Импортируем для работы с категориальными данными 
from google.colab import files # Импортируем для работы с файлами
import matplotlib.pyplot as plt # Импортируем для отрисовки графиков
from gensim.models import word2vec # Импортируем gensim 
import os # Импортируем для работы с файловой системой
import pandas as pd # И мпортируем для работы с DataFrame
import pymorphy2 # Импортируем для работы с леммами
import time # Импортируем для подсчета времени 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Функции для работы с текстом

Преобразуем тесты договоров в списке слов

In [ ]:
# Чтение файла текста из файла, очитска от знаков препинания
def readText(fileName):
  f = open(fileName, 'r') #Открываем наш файл для чтения и считываем из него данные 
  text = f.read() #Записываем прочитанный текст в переменную 
  # Определяем символы для удаления
  delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";"]

  for dS in delSymbols: # Каждый символ в списке символов для удаления
    text = text.replace(dS, " ") # Удаляем, заменяя на пробел

  # Ищем шаблоны и символы в строке и меняем на указанную подстроку
  text = re.sub("[.]", " ", text) 
  text = re.sub(":", " ", text)
  text = re.sub("<", " <", text)
  text = re.sub(">", "> ", text)

  # Метод split разделит текст по пробелам (а их может быть несколько после удаления символов)
  # При помощи метода join запишем все разделенные слова снова в строку
  text = ' '.join(text.split()) 

  text = text.lower() # Переводим текст в нижний регистр
  return text # Возвращаем тексты

# Преобразование исходного текста в список из слов в нормальной форме 
def text2Words(text):
  morph = pymorphy2.MorphAnalyzer() # Создаем экземпляр класса MorphAnalyzer
  words = text.split(' ') # Разделяем текст на пробелы
  words = [morph.parse(word)[0].normal_form for word in words] #Переводим каждое слово в нормалную форму  
  return words # Возвращаем слова


# Чтение данных 

In [ ]:
directory = '/content/drive/My Drive/общая/Договора630/' # Путь к папке с договорами
os.listdir(directory)[250:255] # Выведем названия 5-ти договоров

['dogovor-zaloga-privatizirovannoy-kvartiri.txt',
 'dogovor-zaloga-s-peredachej-predmeta-zaloga-zalogoderzhatelyu.txt',
 'dogovor-zaloga-nedvizhimosti.txt',
 'dogovor-zaloga-imuschestvennih-prav.txt',
 'dogovor-zaloga-imuschestva-kvartiri.txt']

In [ ]:
print('Всего', len(os.listdir(directory)), 'договора.')

Всего 432 договора.


In [ ]:
agreements = [] # Список, в который запишем все наши договоры
for filename in os.listdir(directory): # Проходим по всем файлам в директории договоров
  txt = readText(directory + filename) # Читаем текст договора
  if txt != '': # Если текст не пустой
    agreements.append(readText(directory + filename)) # Преобразуем файл в одну строку и добавляем в agreements

In [ ]:
n = 9
print(os.listdir(directory)[n]) # Выведем название n-ого договора
agreements[n] # Текст n-го договора

3 (3).txt


'договор доверительного управления имуществом г г в лице действующего на основании именуемый в дальнейшем учредитель с одной стороны и в лице действующего на основании именуемый в дальнейшем управляющий с другой стороны именуемые в дальнейшем стороны заключили настоящий договор в дальнейшем договор о нижеследующем 1 предмет договора и другие общие положения 1 1 <s1> согласно настоящему договору учредитель управления передает управляющему на определенный срок имущество в доверительное управление а управляющий обязуется осуществлять управление этим имуществом в интересах учредителя управления </s1> 1 2 передача имущества в доверительное управление не влечет перехода права собственности на него к управляющему 1 3 управляющий вправе совершать в отношении переданного в управление имущества любые юридические и фактические действия в интересах учредителя 1 4 сделки с переданным в управление имуществом управляющий совершает от своего имени указывая при этом что он действует в качестве доверите

In [ ]:
words = [] # Здесь будут храниться все договора в виде списка слов
curTime = time.time() # Засечем текущее время
for i in range(len(agreements)): # Проходимся по всем договорам
  words.append(text2Words(agreements[i])) # Преобразуем очередной договор в список слов и добавляем в words
print('На преобразование ушло:', round(time.time() - curTime, 2), 'с.')

На преобразование ушло: 93.48 с.


In [ ]:
wordsToTest = words[-10:] # Возьмем 10 текстов для финальной проверки обученной нейронной сети 
words = words[:-10] # Для обученающей и проверочной выборок возьмем все тексты, кроме последних 10

In [ ]:
print('Колличество текстов в выборке для финальной проверки в конце ноутбука:', len(wordsToTest)) 
print('Колличество договоров, взятых для обучающей и тестовой выборки:',len(words)) 
print('Колличество слов в первом договоре:', len(words[0])) # Количество слов в первом договоре

Колличество текстов в выборке для финальной проверки в конце ноутбука: 10
Колличество договоров, взятых для обучающей и тестовой выборки: 412
Колличество слов в первом договоре: 1149


In [ ]:
print('Пример исходного текста:')
print(agreements[9][:62], '\n')
print('Тот же текст, представленный в виде списка из слов в начальной форме:')
print(words[5][:10])

Пример исходного текста:
договор доверительного управления имуществом г г в лице действ 

Тот же текст, представленный в виде списка из слов в начальной форме:
['договор', 'аренда', 'автомобиль', 'год', 'год', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать']


# Преобразование текста в индексы: формирование xTrain

In [ ]:
# lower=True - приводим слова к нижнему регистру
# char_level=False - просим токенайзер учитывать слова, а не отдельные символы
tokenizer = Tokenizer(lower=True, filters = '', char_level=False)

tokenizer.fit_on_texts(words) # "Скармливаем" наши тексты, т.е. даём в обработку методу, который соберет словарь частотности
clean_voc = {} 

for item in tokenizer.word_index.items(): #Преобразуем полученный список 
  clean_voc[item[0]] = item[1] # В словарь, меняя местами элементы полученного кортежа 

In [ ]:
print('Словарь всех слов по частоте:') 
print(clean_voc, '\n')
print('Длинна словаря:', len(clean_voc))

tag = '<s1>'
print('Индекс тега', tag, ':' ,clean_voc[tag])

Словарь всех слов по частоте:
{'в': 1, 'договор': 2, 'и': 3, '2': 4, '1': 5, 'по': 6, 'сторона': 7, 'с': 8, 'на': 9, 'настоящий': 10, '3': 11, '<s1>': 12, '</s1>': 13, 'не': 14, '4': 15, 'или': 16, 'он': 17, 'за': 18, 'случай': 19, 'о': 20, '5': 21, '•': 22, '<s3>': 23, '</s3>': 24, 'право': 25, 'работа': 26, 'срок': 27, 'плата': 28, 'от': 29, '<s4>': 30, '</s4>': 31, 'год': 32, 'при': 33, '6': 34, 'лицо': 35, 'имущество': 36, 'арендатор': 37, 'другой': 38, 'быть': 39, 'день': 40, 'для': 41, 'условие': 42, '7': 43, 'если': 44, 'к': 45, 'дальнейший': 46, 'который': 47, 'обязательство': 48, 'действующий': 49, 'а': 50, 'страховой': 51, 'сумма': 52, 'указанный': 53, 'порядок': 54, 'весь': 55, 'законодательство': 56, 'арендодатель': 57, 'работник': 58, 'они': 59, 'действие': 60, 'счёт': 61, 'свой': 62, 'размер': 63, 'соответствие': 64, 'именовать': 65, 'из': 66, 'мочь': 67, 'предусмотренный': 68, 'один': 69, 'обязать': 70, 'страховщик': 71, 'исполнение': 72, 'основание': 73, 'страхователь':

In [ ]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
tok_agreem = tokenizer.texts_to_sequences(words) # Обучающие тесты в индексы

print("Взглянем на фрагмент обучающего текста:")
print("В виде оригинального текста:              ", words[9][:20])
print("Он же в виде последовательности индексов: ", tok_agreem[9][:20], '\n')

Взглянем на фрагмент обучающего текста:
В виде оригинального текста:               ['договор', 'доверительный', 'управление', 'имущество', 'год', 'год', 'в', 'лицо', 'действующий', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'учредитель', 'с', 'один', 'сторона', 'и', 'в']
Он же в виде последовательности индексов:  [2, 133, 122, 36, 32, 32, 1, 35, 49, 9, 73, 65, 1, 46, 215, 8, 69, 7, 3, 1] 



# Формирование yTrain

Каждому индексу (слову) из договора необходимо поставить в соответсвие вектор из 6-ти элементов, соответствующих тегам категорий

In [ ]:
# Собираем список индексов и dummy encoded вектора
def get01XSamples(tok_agreem, tags_index):
  tags01 = [] # Список для тегов
  indexes = [] # Здесь будут лежать индексы
 
  for agreement in tok_agreem: # Проходимся по каждому договору-списку
    tag_place = [0, 0, 0, 0, 0, 0] # Создаем вектор [0,0,0,0,0,0]
    for ex in agreement: # Проходимся по каждому слову договора
        if ex in tags_index: # Смотрим, если индекс оказался нашим тегом
          place = np.argwhere(tags_index==ex) # Записываем под каким местом лежит этот тег в своем списке
          if len(place)!=0: # Проверяем, чтобы тег действительно был
            if place[0][0]<6: # Первые шесть тегов в списке - открывающие
              tag_place[place[0][0]] = 1    # Поэтому ставим 1
            else: 
              tag_place[place[0][0] - 6] = 0  # Остальные теги закрывающие, так что меняем на ноль
        else:          
          tags01.append(tag_place.copy()) # Расширяем наш список с каждой итерацией. Получаем в конце длинный список из всех тегов в одном 
          indexes.append(ex) # Докидываем индекс слова в список индексов

  return indexes, tags01

# Получение списка слов из индексов
def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items())) # Берем пары значений всего словаря и размечаем наоборот, т.е. value:key
  words = [reverse_word_map.get(letter) for letter in x] # Вытаскиваем по каждому ключу в список
  return words # Возвращаем полученный текст

In [ ]:
# s1 Условия
# s2 Запреты
# s3 Стоимость, всё про цены и деньги
# s4 Всё про сроки
# s5 Неустойка
# s6 Всё про адреса и геолокации

tags_index = ['<s' + str(i) + '>' for i in range(1, 7)] # Получаем список открывающих тегов
closetags = ['</s' + str(i) + '>' for i in range(1, 7)] # Получаем список закрывающих тегов
tags_index.extend(closetags) # Объединяем все теги

tags_index = np.array([clean_voc[i] for i in tags_index]) # Получаем из словаря частотности индексы всех тегов
print('Индексы тегов:', tags_index)

Индексы тегов: [ 12 348  23  30 115 272  13 356  24  31 116 263]


In [ ]:
xData, yData = get01XSamples(tok_agreem,tags_index) # Распознаем теги и создаем список с ними, с индексами
decoded_text = reverseIndex(clean_voc, xData) # Для создания списков с embedding-ами сначала преобразуем список индексов обратно в слова

In [ ]:
print('Длина xData:', len(xData))
print('Длина yData:', len(yData))

print('Посмотрим на исходный набор слов: ', words[0][50:80])
print('Посмотрим на декодированный текст:', decoded_text[50:80])
print('Посмотрим на часть из xData:     ', xData[50:80])
print('Посмотрим на часть из yData:     ', yData[50:80])

Длина xData: 411870
Длина yData: 411870
Посмотрим на исходный набор слов:  ['1', 'предмет', 'договор', '1', '1', '<s1>', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '</s1>', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по']
Посмотрим на декодированный текст: ['1', 'предмет', 'договор', '1', '1', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор']
Посмотрим на часть из xData:      [5, 107, 2, 5, 5, 6, 10, 2, 216, 112, 925, 571, 119, 17, 238, 3, 541, 255, 571, 1, 409, 5, 4, 238, 297, 9, 155, 6, 10, 2]
Посмотрим на часть из yData:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0]

# Разделение выборки на окна

In [ ]:
# Формируем выборку из индексов
def getSetFromIndexes(wordIndexes, xLen, step): 
  xBatch = [] # Лист для фрагментов текста
  wordsLen = len(wordIndexes) # Определяем длинну текста
  index = 0 # Задаем стартовый индекс
  
  while (index + xLen <= wordsLen): # Пока сумма индекса с длинной фрагмента меньше или равна числу слов в выборке
    xBatch.append(wordIndexes[index:index+xLen]) # Добавляем X в лист фразментов текста
    index += step # Сдвигаемся на step

  return xBatch # Лист для фрагментов текста

In [ ]:
xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 300 # Количество измерений для векторного пространства

In [ ]:
# Генерируем наборы с заданными параметрами окна
xTrain = getSetFromIndexes(decoded_text, xLen, step) # Последовательность из xLen слов
yTrain = getSetFromIndexes(yData, xLen, step) # Последовательность из xLen-тегов

In [ ]:
print('Длинна xTrain:', len(xTrain))
print('Длинна yTrain:', len(yTrain))
print('Длинна примера из xTrain:',len(xTrain[0]))
print('Длинна примера из yTrain:',len(yTrain[0]), '\n')
print('Пример xTrain', xTrain[0])
print('Пример yTrain', yTrain[0], '\n')

print('Первый пример xTrain:', xTrain[0][step-5:step+5])
print('Второй пример xTrain:', xTrain[1][:10])

Длинна xTrain: 13721
Длинна yTrain: 13721
Длинна примера из xTrain: 256
Длинна примера из yTrain: 256 

Пример xTrain ['договор', 'безвозмездный', 'хранение', 'до', 'востребование', 'год', 'год', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'хранитель', 'с', 'один', 'сторона', 'и', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'поклажедатель', 'с', 'другой', 'сторона', 'именовать', 'в', 'дальнейший', 'сторона', 'заключить', 'настоящий', 'договор', 'в', 'дальнейший', 'договор', 'о', 'нижеследующий', '1', 'предмет', 'договор', '1', '1', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор', 'следующий', 'вещь', 'далее', 'вещь', '1', '3', 'при', 'передача', 'ве

# Функции для создания моделей

## Функция подсчета ошибки

In [ ]:
# Функция, которая смотрит на пересечение областей. Нужна для accuracy
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

## **Conv1D**

In [ ]:
# Функция для создания линейной cnonv1d-сети
def create_Conv1d(xLen, embeddingSize): 
  text_input_layer = Input((xLen,embeddingSize)) 
  text_layer = Conv1D(16, 3, padding='same',activation='relu')(text_input_layer)
  text_layer = Conv1D(16, 3, padding='same',activation='relu')(text_layer)
  text_layer = Conv1D(16, 3,padding='same', activation='relu')(text_layer) 
  text_layer = Conv1D(yTrainGENSIM.shape[-1], 3, padding='same',activation='relu')(text_layer)
  model = Model(text_input_layer, text_layer)
  model.compile(optimizer=Adadelta(),
                    loss='categorical_crossentropy',
                    metrics=[dice_coef])
  return model

## **PSPnet**

In [ ]:
# Функция для создания PSPnet-сети
def create_PSPNet(conv_size = 64, num_classes = 6, input_shape = (30, 300)):

    # У кераса нет Conv1DTranspose, так что прописываем вручную. Нужен для разворота после maxpooling
    def Conv1DTranspose(input_tensor, filters, kernel_size=1, strides=2, padding='same'):
        """
        input_tensor: входной тензор (batch_size, time_steps, dims)
        filters: int, output dimension, выходной тензор будет иметь размер (batch_size, time_steps, filters)
        kernel_size: размер ядра свертки 
        strides: int, шаг ядра
        padding: 'same' | действительный
        """
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x
        
    img_input = Input(input_shape)

    # Block 1
    x = Conv1D(conv_size, 3, padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv1D(conv_size, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x_mp_2 = MaxPooling1D(2)(x)
    x_mp_4 = MaxPooling1D(4)(x)
    x_mp_8 = MaxPooling1D(8)(x)
    x_mp_16 = MaxPooling1D(16)(x)
    x_mp_32 = MaxPooling1D(32)(x)

    x_mp_2 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_2)
    x_mp_2 = Dropout(0.5)(x_mp_2)

    x_mp_4 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_4)
    x_mp_4 = Dropout(0.5)(x_mp_4)

    x_mp_8 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_8)
    x_mp_8 = Dropout(0.5)(x_mp_8)

    x_mp_16 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_16)
    x_mp_16 = Dropout(0.5)(x_mp_16)

    x_mp_32 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_32)
    x_mp_32 = Dropout(0.5)(x_mp_32) 

    x_mp_2 = Conv1DTranspose(x_mp_2,conv_size, 2)
    x_mp_2 = Activation('relu')(x_mp_2)

    x_mp_4 = Conv1DTranspose(x_mp_4,conv_size, 4, strides=4)
    x_mp_4 = Activation('relu')(x_mp_4)

    x_mp_8 = Conv1DTranspose(x_mp_8,conv_size, 8, strides=8)
    x_mp_8 = Activation('relu')(x_mp_8)

    x_mp_16 = Conv1DTranspose(x_mp_16,conv_size, 16, strides=16)
    x_mp_16 = Activation('relu')(x_mp_16)

    x_mp_32 = Conv1DTranspose(x_mp_32,conv_size, 32, strides=32)
    x_mp_32 = Activation('relu')(x_mp_32)


    fin = concatenate([img_input, x_mp_2, x_mp_4, x_mp_8, x_mp_16, x_mp_32])    
    fin = Conv1D(conv_size, 3, padding='same')(fin)
    fin = BatchNormalization()(fin)
    fin = Activation('relu')(fin)
    fin = Conv1D(conv_size, 3, padding='same')(fin)
    fin = BatchNormalization()(fin)
    fin = Activation('relu')(fin)

    fin = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(fin)

    model = Model(img_input, fin)
    model.compile(optimizer=Adadelta(lr=0.001),
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
    
    return model

##**UNET**

In [ ]:
#Функция для создания сети UNET 
def create_unet(k = 1, num_classes = 6, input_shape= (30, 300)):

    # У кераса нет Conv1DTranspose, так что прописываем вручную. Нужен для разворота после maxpooling
    def Conv1DTranspose(input_tensor, filters, kernel_size=1, strides=2, padding='same'):
        """
        input_tensor: входной тензор (batch_size, time_steps, dims)
        filters: int, output dimension, выходной тензор будет иметь размер (batch_size, time_steps, filters)
        kernel_size: размер ядра свертки 
        strides: int, шаг ядра
        padding: 'same' | действительный
        """
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x
        
    img_input = Input(input_shape) 

    # Block 1
    x = Conv1D(64 * k , 3, padding='same')(img_input) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    x = MaxPooling1D()(block_1_out)

    # Block 2
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    x = MaxPooling1D()(block_2_out)

    # Block 3
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # x = block_3_out
    x = MaxPooling1D()(block_3_out)

    # Block 4
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(x, 256, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(x, 128, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(x, 64, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='softmax', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

# Создание xTrain и  yTrain при помощи метода Word2wec GENSIM

In [ ]:
# Создаем выборки
def getSets(model, senI, tagI):
  xVector = [] # Здесь будет лежать embedding представление каждого из индексов
  tmp = [] # Временный список
  for text in senI: # Проходимся по каждому тексту-списку
    tmp=[]
    for word in text: # Проходимся по каждому слову в тексте-списке
      tmp.append(model[word]) 

    xVector.append(tmp)

  return np.array(xVector), np.array(tagI)

In [ ]:
# Передаем в word2vec списки списков слов для обучения
# size = embeddingSize - размер эмбеддинга
# window = 10 - минимальное расстояние между словами в эмбеддинге 
# min_count = 1 - игнорирование всех слов с частотой, меньше, чем 1
# workers = 10 - число потоков обучения эмбеддинга
# iter = 10 - число эпох обучения эмбеддинга

modelGENSIM = word2vec.Word2Vec(xTrain, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10)

In [ ]:
modelGENSIM['слово'] # Пример преобразования слова в вектор при помощи GENSIM

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-7.65841961e-01, -4.56496865e-01,  9.90821242e-01, -4.71479923e-01,
        6.67062700e-01,  1.68664205e+00, -2.79778862e+00, -5.44480622e-01,
       -2.00066194e-01,  8.90309632e-01,  1.77705979e+00,  3.22440773e-01,
       -1.25987157e-01, -6.46479502e-02, -9.91665661e-01, -2.43205830e-01,
        3.38405085e+00,  1.72131884e+00,  1.05887520e+00,  1.36988878e+00,
        1.68812001e+00,  1.34152341e+00,  1.14523447e+00, -1.74783602e-01,
        9.36550558e-01,  3.39489849e-03,  1.08459663e+00, -4.71059024e-01,
        1.46760240e-01, -3.76989186e-01, -1.10753968e-01,  8.36523354e-01,
       -9.48799014e-01,  1.88985503e+00, -1.19127858e+00, -9.71118450e-01,
       -2.15863436e-01,  2.84974247e-01, -4.83836055e-01, -9.16195691e-01,
        2.82631546e-01,  9.35399294e-01,  5.09415329e-01,  3.26404840e-01,
       -8.48372161e-01,  5.57956815e-01, -1.76185417e+00, -1.59252539e-01,
        7.54542887e-01,  4.02996570e-01, -2.09430069e-01, -7.34804392e-01,
        5.29252231e-01,  

In [ ]:
xTrainGENSIM, yTrainGENSIM = getSets(modelGENSIM, xTrain, yTrain)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
print('Размерность полученного xTrain:', xTrainGENSIM.shape)
print('Размерность полученного yTrain:', yTrainGENSIM.shape)

Размерность полученного xTrain: (13721, 256, 300)
Размерность полученного yTrain: (13721, 256, 6)


# Сохранение массивов из оперативной памяти в память runtime 

**Перед перезагрузкой сессии**

In [ ]:
np.save('xTrainGENSIM.npy', xTrainGENSIM) #Сохраняем массивы в память колаба
np.save('yTrainGENSIM.npy', yTrainGENSIM) #Сохраняем массивы в память колаба

**После перезагрузки сессии**

In [ ]:
xTrainGENSIM = np.load('xTrainGENSIM.npy') #Загружаем массивы из памяти колаба
yTrainGENSIM = np.load('yTrainGENSIM.npy') #Загружаем массивы из памяти колаба

# Обучение сетей

## **Линейная Conv1d-сеть**

In [ ]:
model_conv1d = create_Conv1d(xLen, embeddingSize) # Создаем простую сеть
model_conv1d.summary() # Смотрим полученную архитектуру 

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 300)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 256, 16)           14416     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 256, 16)           784       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 256, 16)           784       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 256, 6)            294       
Total params: 16,278
Trainable params: 16,278
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model_conv1d.fit(xTrainGENSIM, yTrainGENSIM, epochs=40, batch_size=100, validation_split = 0.2) # Скармливаем модели полученные вектора

Epoch 1/40
110/110 [==============================] - 3s 23ms/step - loss: 6.8950 - dice_coef: 0.1121 - val_loss: 7.0605 - val_dice_coef: 0.1147
Epoch 2/40
110/110 [==============================] - 2s 16ms/step - loss: 6.8597 - dice_coef: 0.1130 - val_loss: 7.0344 - val_dice_coef: 0.1154
Epoch 3/40
110/110 [==============================] - 2s 16ms/step - loss: 6.8358 - dice_coef: 0.1136 - val_loss: 7.0161 - val_dice_coef: 0.1159
Epoch 4/40
110/110 [==============================] - 2s 16ms/step - loss: 6.8183 - dice_coef: 0.1141 - val_loss: 6.9984 - val_dice_coef: 0.1163
Epoch 5/40
110/110 [==============================] - 2s 16ms/step - loss: 6.7962 - dice_coef: 0.1146 - val_loss: 6.9785 - val_dice_coef: 0.1168
Epoch 6/40
110/110 [==============================] - 2s 16ms/step - loss: 6.7763 - dice_coef: 0.1152 - val_loss: 6.9558 - val_dice_coef: 0.1174
Epoch 7/40
110/110 [==============================] - 2s 16ms/step - loss: 6.7465 - dice_coef: 0.1159 - val_loss: 6.9273 - val_dic

## **PSPnet**

### Стандартная архитектура

In [ ]:
model_b_PSPnet = create_PSPNet(input_shape=(xLen, embeddingSize))
model_b_PSPnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 256, 64)      57664       input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 64)      256         conv1d_4[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 64)      0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [ ]:
history = model_b_PSPnet.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=32, validation_split = 0.2)

Epoch 1/20
343/343 [==============================] - 6s 17ms/step - loss: 1.3341 - dice_coef: 0.1959 - val_loss: 1.3132 - val_dice_coef: 0.2059
Epoch 2/20
343/343 [==============================] - 5s 14ms/step - loss: 1.3095 - dice_coef: 0.2013 - val_loss: 1.2929 - val_dice_coef: 0.2111
Epoch 3/20
343/343 [==============================] - 5s 14ms/step - loss: 1.2842 - dice_coef: 0.2073 - val_loss: 1.2709 - val_dice_coef: 0.2164
Epoch 4/20
343/343 [==============================] - 5s 14ms/step - loss: 1.2589 - dice_coef: 0.2136 - val_loss: 1.2490 - val_dice_coef: 0.2218
Epoch 5/20
343/343 [==============================] - 5s 14ms/step - loss: 1.2341 - dice_coef: 0.2200 - val_loss: 1.2275 - val_dice_coef: 0.2274
Epoch 6/20
343/343 [==============================] - 5s 14ms/step - loss: 1.2099 - dice_coef: 0.2266 - val_loss: 1.2068 - val_dice_coef: 0.2331
Epoch 7/20
343/343 [==============================] - 5s 14ms/step - loss: 1.1864 - dice_coef: 0.2333 - val_loss: 1.1864 - val_dic

### 256 нейронов на ядре свертки

In [ ]:
model_b_PSPnet = create_PSPNet(conv_size = 256, input_shape=(xLen, embeddingSize))
model_b_PSPnet.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_51 (Conv1D)              (None, 256, 256)     230656      input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_31 (BatchNo (None, 256, 256)     1024        conv1d_51[0][0]                  
__________________________________________________________________________________________________
activation_46 (Activation)      (None, 256, 256)     0           batch_normalization_31[0][0]     
____________________________________________________________________________________________

In [ ]:
history = model_b_PSPnet.fit(xTrainGENSIM, yTrainGENSIM, epochs=60, batch_size=32, validation_split = 0.2)

Epoch 1/60
343/343 [==============================] - 15s 45ms/step - loss: 1.1654 - dice_coef: 0.2433 - val_loss: 1.1849 - val_dice_coef: 0.2422
Epoch 2/60
343/343 [==============================] - 15s 43ms/step - loss: 1.1113 - dice_coef: 0.2601 - val_loss: 1.1352 - val_dice_coef: 0.2579
Epoch 3/60
343/343 [==============================] - 15s 43ms/step - loss: 1.0627 - dice_coef: 0.2765 - val_loss: 1.0911 - val_dice_coef: 0.2730
Epoch 4/60
343/343 [==============================] - 15s 43ms/step - loss: 1.0197 - dice_coef: 0.2922 - val_loss: 1.0526 - val_dice_coef: 0.2872
Epoch 5/60
343/343 [==============================] - 15s 43ms/step - loss: 0.9820 - dice_coef: 0.3074 - val_loss: 1.0192 - val_dice_coef: 0.3005
Epoch 6/60
343/343 [==============================] - 15s 43ms/step - loss: 0.9498 - dice_coef: 0.3213 - val_loss: 0.9907 - val_dice_coef: 0.3128
Epoch 7/60
343/343 [==============================] - 15s 43ms/step - loss: 0.9226 - dice_coef: 0.3341 - val_loss: 0.9659 - 

### 512 нейронов на ядре свертки

In [ ]:
model_b_PSPnet = create_PSPNet(conv_size = 512, input_shape=(xLen, embeddingSize))
model_b_PSPnet.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_61 (Conv1D)              (None, 256, 512)     461312      input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_35 (BatchNo (None, 256, 512)     2048        conv1d_61[0][0]                  
__________________________________________________________________________________________________
activation_55 (Activation)      (None, 256, 512)     0           batch_normalization_35[0][0]     
____________________________________________________________________________________________

In [ ]:
history = model_b_PSPnet.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=64, validation_split = 0.2)

ResourceExhaustedError: ignored

## **UNET**

### Стандартная архитектура

In [ ]:
model_b_UNET = create_unet(input_shape=(xLen, embeddingSize))
model_b_UNET.summary()

In [ ]:
history = model_b_UNET.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=64, validation_split = 0.2) # Скармливаем модели изображения, разделенные по 3 классам

# **Проверка работы сети**

In [ ]:
# Функция, выводящая точность распознавания каждой категории отдельно
def recognizeSet(tagI, pred, tags, length, value):
  total=0

  for j in range(6): # общее количество тегов
    correct=0
    for i in range(len(tagI)): # проходимся по каждому списку списка тегов
      for k in range(length): # проходимся по каждому тегу
        if tagI[i][k][j]==(pred[i][k][j]>value).astype(int): # если соответствующие индексы совпадают, значит сеть распознала верно
          correct+=1 
    print("Сеть распознала категорию '{}' на {}%".format(tags[j],round(100*correct/(len(tagI)*length), 2)))
    total += 100 * correct / (len(tagI)*length)
  print("Cредняя точность {}%".format(total/6))

In [ ]:
model = model_b_UNET 

In [ ]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
tok_agreemTest = tokenizer.texts_to_sequences(wordsToTest) # Обучающие тесты в индексы

print("Взглянем на фрагмент тестового текста:")
print("В виде оригинального текста:              ", wordsToTest[4][:20])
print("Он же в виде последовательности индексов: ", tok_agreemTest[4][:20], '\n')

Взглянем на фрагмент тестового текста:
В виде оригинального текста:               ['трудовой', 'контракт', 'с', 'специалист', 'год', 'год', 'в', 'лицо', 'действующий', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'работодатель', 'с', 'один', 'сторона', 'и', 'гражданин']
Он же в виде последовательности индексов:  [90, 330, 8, 1922, 32, 32, 1, 35, 49, 9, 73, 65, 1, 46, 136, 8, 69, 7, 3, 270] 



In [ ]:
xDataTest, yDataTest = get01XSamples(tok_agreemTest,tags_index) # Распознаем теги и создаем список с ними, с индексами
decoded_text = reverseIndex(clean_voc, xDataTest) # Для создания списков с embedding-ами сначала преобразуем список индексов обратно в слова

In [ ]:
print('Длина xDataTest:', len(xDataTest))
print('Длина yDataTest:', len(yDataTest))

print('Посмотрим на исходный набор слов: ', wordsToTest[0][50:80])
print('Посмотрим на декодированный текст:', decoded_text[50:80])
print('Посмотрим на часть из xDataTest:     ', xDataTest[50:80])
print('Посмотрим на часть из yDataTest:     ', yDataTest[50:80])

Длина xDataTest: 12647
Длина yDataTest: 12647
Посмотрим на исходный набор слов:  ['1', 'предмет', 'договор', '1', '1', 'работник', 'приниматься', 'на', 'работа', 'на', 'предприятие', 'на', 'должность', 'руководитель', 'служба', 'реклама', 'и', 'маркетинг', 'для', 'выполнение', 'трудовой', 'обязанность', 'связанный', 'с', 'руководство', 'указанный', 'служба', 'планирование', 'и', 'стимулирование']
Посмотрим на декодированный текст: ['1', 'предмет', 'договор', '1', '1', 'работник', 'приниматься', 'на', 'работа', 'на', 'предприятие', 'на', 'должность', 'руководитель', 'служба', 'реклама', 'и', 'маркетинг', 'для', 'выполнение', 'трудовой', 'обязанность', 'связанный', 'с', 'руководство', 'указанный', 'служба', 'планирование', 'и', 'стимулирование']
Посмотрим на часть из xDataTest:      [5, 107, 2, 5, 5, 58, 1034, 9, 26, 9, 470, 9, 536, 820, 730, 1489, 3, 4057, 41, 101, 90, 75, 295, 8, 1045, 53, 730, 2605, 3, 5671]
Посмотрим на часть из yDataTest:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]

In [ ]:
xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 300 # Количество измерений для векторного пространства

# Генерируем наборы с заданными параметрами окна
xTest = getSetFromIndexes(decoded_text, xLen, step) # Последовательность из xLen слов
yTest = getSetFromIndexes(yDataTest, xLen, step) # Последовательность из xLen-тегов

In [ ]:
print('Длинна xTest:', len(xTest))
print('Длинна yTest:', len(yTest))
print('Длинна примера из xTest:',len(xTest[0]))
print('Длинна примера из yTrain:',len(yTest[0]), '\n')
print('Пример xTest', xTest[0])
print('Пример yTest', yTest[0], '\n')

print('Первый пример xTest:', xTest[0][step-5:step+5])
print('Второй пример xTest:', xTest[1][:10])

Длинна xTest: 414
Длинна yTest: 414
Длинна примера из xTest: 256
Длинна примера из yTrain: 256 

Пример xTest ['трудовой', 'контракт', 'с', 'руководитель', 'служба', 'реклама', 'и', 'маркетинг', 'год', 'год', 'в', 'лицо', 'действующий', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'предприятие', 'с', 'один', 'сторона', 'и', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'работник', 'с', 'другой', 'сторона', 'именовать', 'в', 'дальнейший', 'сторона', 'заключить', 'настоящий', 'договор', 'в', 'дальнейший', 'договор', 'о', 'нижеследующий', '1', 'предмет', 'договор', '1', '1', 'работник', 'приниматься', 'на', 'работа', 'на', 'предприятие', 'на', 'должность', 'руководитель', 'служба', 'реклама', 'и', 'маркетинг', 'для', 'выполнение', 'трудовой', 'обязанность', 'связанный', 'с', 'руководство', 'указанный', 'служба', 'планирование', 'и', 'стимулирование', 'сбыт', 'изучение', 'и', 'рынок', 'реклама', 'предприятие', 'он', 'в

In [ ]:
# Передаем в word2vec списки списков слов для обучения
# size = embeddingSize - размер эмбеддинга
# window = 10 - расстояние между текущим и прогнозируемым словом в предложении
# min_count = 1 - игнорирование всех слов с частотой, меньше, чем 1
# workers = 10 - число потоков обучения эмбеддинга
# iter = 10 - число эпох обучения эмбеддинга

modelGENSIM = word2vec.Word2Vec(xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10)

In [ ]:
xTestGENSIM, yTestGENSIM = getSets(modelGENSIM, xTest, yTest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
print('Размерность полученного xTrain:', xTestGENSIM.shape)
print('Размерность полученного yTrain:', yTestGENSIM.shape)

Размерность полученного xTrain: (414, 256, 300)
Размерность полученного yTrain: (414, 256, 6)


In [ ]:
tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
# s1 Условия
# s2 Запреты
# s3 Стоимость, всё про цены и деньги
# s4 Всё про сроки
# s5 Неустойка
# s6 Всё про адреса и геолокации

In [ ]:
pred = model.predict(xTestGENSIM) # сделаем предсказание

In [ ]:
recognizeSet(yTestGENSIM, pred, tags, xLen, 0.999)

Сеть распознала категорию 'S1' на 64.39%
Сеть распознала категорию 'S2' на 97.63%
Сеть распознала категорию 'S3' на 86.18%
Сеть распознала категорию 'S4' на 87.36%
Сеть распознала категорию 'S5' на 93.96%
Сеть распознала категорию 'S6' на 99.96%
Cредняя точность 88.24413747987118%
